In [ ]:
import matplotlib.pyplot as plt
import numpy as np

import lsst.daf.butler as dB
import lsst.cp.verify.notebooks.utils as utils
import lsst.afw.display as afwDisplay

In [ ]:
# This cell contains parameters that can be automatically set via the papermill package.
# Examples:
#   papermill --prepare-only -p calibType newBias -p cameraName LSSTCam <input> <output>
#   papermill -p interactive False <input> <output>
interactive = True

# Which repository to use.
repository = '/repo/main/'

# Which calibration type to analyse.
calibType = 'ptc'
detectorNumber = 0

# Which camera the calibration is for.
cameraName = 'LATISS'

# Which display to use.
displayBackend = 'astrowidgets'

# Which collections the calibration was constructed in.
genCollection = 'u/czw/DM-28920/ptcGen.20210723Ya'
genCollection2 = 'u/czw/DM-28920/ptcGen.20210723Yb'

# Which collections containing the original outputs.
originalGenCollection = 'u/czw/DM-28920/ptcGen.20210722a'
originalGenCollection2 = 'u/czw/DM-28920/ptcGen.20210722b'

# Which collection contains the linearity output.
linearityCollection = 'u/czw/DM-28920/linearityGen.20210721a'

In [ ]:
# Get butler and camera
butler = dB.Butler(repository, collections=[genCollection, genCollection2, 
                                            originalGenCollection, 
                                            originalGenCollection2])
camera = butler.get('camera', instrument=cameraName)

In [ ]:
# Instantiate separate butlers to ensure the correct dataset is found.
butler = dB.Butler(repository, collections=[genCollection])
ptcA = butler.get('ptc', instrument=cameraName, detector=detectorNumber)
print(ptcA.getMetadata()['CALIBDATE'])

butler = dB.Butler(repository, collections=[genCollection2])
ptcB = butler.get('ptc', instrument=cameraName, detector=detectorNumber)
print(ptcB.getMetadata()['CALIBDATE'])

butler = dB.Butler(repository, collections=[originalGenCollection])
ptcOA = butler.get('ptc', instrument=cameraName, detector=detectorNumber)
print(ptcOA.getMetadata()['CALIBDATE'])

butler = dB.Butler(repository, collections=[originalGenCollection2])
ptcOB = butler.get('ptc', instrument=cameraName, detector=detectorNumber)
print(ptcOB.getMetadata()['CALIBDATE'])

In [ ]:
# Get linearizer
linearizer = butler.get('linearity', instrument=cameraName, detector=detectorNumber, 
                        collections=linearityCollection)

In [ ]:
# View plots.
viewPlots = interactive
if viewPlots:
    for amp in ptcA.ampNames:
        plt.scatter(ptcA.rawMeans[amp], ptcA.rawVars[amp], marker='o', label="PTCA")
        plt.scatter(ptcB.rawMeans[amp], ptcB.rawVars[amp], marker='o', label="PTCA")

        plt.scatter(ptcA.finalMeans[amp], ptcA.finalVars[amp], marker='v', label="PTCA")
        plt.scatter(ptcB.finalMeans[amp], ptcB.finalVars[amp], marker='^', label='PTCB')
    
        plt.scatter(ptcOA.finalMeans[amp], ptcOA.finalVars[amp], marker='<', label="PTCA")
        plt.scatter(ptcOB.finalMeans[amp], ptcOB.finalVars[amp], marker='>', label='PTCB')

        plt.show()
    
        continueDisplay, skipNumber = utils.interactiveBlock(f"{calibType} {detector.getName()}", 
                                                             {})
        if continueDisplay is False:
            break

In [ ]:
# Non interactive plots
plt.scatter(np.array(list(ptcA.gain.values())), 
            np.array(list(ptcA.gain.values())) - np.array(list(ptcB.gain.values())), 
            marker='+', label="PTCA")
plt.scatter(np.array(list(ptcA.noise.values())), 
            np.array(list(ptcA.noise.values())) - np.array(list(ptcB.noise.values())),
            marker='x', label='PTCB')
plt.show()
print("A Gain:", [f"{x:.3f}" for x in ptcA.gain.values()])
print("A RN:  ", [f"{x:.3f}" for x in ptcA.noise.values()])

print("B Gain:", [f"{x:.3f}" for x in ptcB.gain.values()])
print("B RN:  ", [f"{x:.3f}" for x in ptcB.noise.values()])